By deploying or using this software you agree to comply with the [AI Hub Terms of Service]( https://aihub.cloud.google.com/u/0/aihub-tos) and the [Google APIs Terms of Service](https://developers.google.com/terms/). To the extent of a direct conflict of terms, the AI Hub Terms of Service will control.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GoogleCloudPlatform/ai-platform-samples/blob/main/notebooks/samples/aihub/tabular_anomaly_detection/tabular_anomaly_detection.ipynb)

# Overview

This notebook provides an example workflow of using the [Tabular Anomaly Detection ML container](https://aihub.cloud.google.com/u/0/p/products%2F6427563a-f2a8-4f9e-a104-a4dbc95d4e3e) for training a classification ML model.

### Dataset

The notebook uses the [kddcup 99 dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_kddcup99.html) with SA subset. It consists of 41 features with a mix of categorical and numerical data types. The target is a binary variable with 0 or 1 to indicate a normal or anomalous sample. The ratio of normal to anomalous samples is approximately 29 to 1.

### Objective

The goal of this notebook is to go through a common training workflow:
- Create a dataset
- Train an ML model using the [AI Platform Training](https://cloud.google.com/ai-platform/training/docs) service
- Monitor the training job with [TensorBoard](https://www.tensorflow.org/tensorboard)
- Identify if the model was trained successfully by looking at the generated "Run Report"
- Deploy the model for serving using the [AI Platform Prediction](https://cloud.google.com/ai-platform/prediction/docs/overview) service
- Use the endpoint for online predictions
- Interactively inspect the deployed ML model with the [What-If Tool](https://pair-code.github.io/what-if-tool/) 

### Costs 

This tutorial uses billable components of Google Cloud Platform (GCP):

* Cloud AI Platform
* Cloud Storage

Learn about [Cloud AI Platform
pricing](https://cloud.google.com/ml-engine/docs/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or AI Platform Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

2. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

3. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3.

4. Activate that environment and run `pip install jupyter` in a shell to install
   Jupyter.

5. Run `jupyter notebook` in a shell to launch Jupyter.

6. Open this notebook in the Jupyter Notebook Dashboard.

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project.](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AI Platform APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

In [ ]:
PROJECT_ID = "[your-project-id]" #@param {type:"string"}
! gcloud config set project $PROJECT_ID

### Authenticate your GCP account

**If you are using AI Platform Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the GCP Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. From the **Service account** drop-down list, select **New service account**.

3. In the **Service account name** field, enter a name.

4. From the **Role** drop-down list, select
   **Machine Learning Engine > AI Platform Admin** and
   **Storage > Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()

# If you are running this notebook locally, replace the string below with the
# path to your service account key and run this cell to authenticate your GCP
# account.
else:
  %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

You need to have a "workspace" bucket that will hold the dataset and the output from the ML Container. Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets. 

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Cloud AI Platform services are available](https://cloud.google.com/ml-engine/docs/tensorflow/regions). You may not use a Multi-Regional Storage bucket for training with AI Platform.

In [ ]:
BUCKET_NAME = "[your-bucket-name]" #@param {type:"string"}
REGION = 'us-central1' #@param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

## PIP Install Packages and dependencies

In [ ]:
! pip install witwidget

### Import libraries and define constants

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from IPython.core.display import HTML
from googleapiclient import discovery

## Create a dataset

In [17]:
# load kddcup99 dataset with SA subset
kddcup99 = datasets.fetch_kddcup99(subset='SA')
data = pd.DataFrame(kddcup99.data).infer_objects()
data.columns = ["f{}".format(c) for c in data.columns]

# replace categorical columns with their 1-hot vectors
for categorical_column in data.columns[data.dtypes=='object']:
    data[categorical_column] = data[categorical_column].str.decode('utf-8')
    oh_feature = pd.get_dummies(data[categorical_column])
    oh_feature.columns = ["{}{}".format(categorical_column, c) for c in oh_feature.columns]
    data = pd.concat([data, oh_feature], axis=1)
    del data[categorical_column]
    
feature_names = data.columns
# add a binary anomly variable
data['target'] = (kddcup99.target != b'normal.').astype(np.int32)

# split to training and validation subsets
training, validation = train_test_split(data, test_size=.2, stratify=data['target'], random_state=0)

# standardization
training_targets = training.pop('target')
validation_targets = validation.pop('target')

data_mean = training.mean(axis=0)
data_std = training.std(axis=0)

training = (training - data_mean) / data_std
training = training.fillna(0.)
training['target'] = training_targets

validation = (validation - data_mean) / data_std
validation = validation.fillna(0.)
validation['target'] = validation_targets

print('Training data head')
display(training.head())

training_data = os.path.join('gs://', BUCKET_NAME, 'data/train.csv')
validation_data = os.path.join('gs://', BUCKET_NAME, 'data/valid.csv')

print('Copy the data in bucket ...')
with tf.io.gfile.GFile(training_data, 'w') as f:
  training.to_csv(f, index=False)
with tf.io.gfile.GFile(validation_data, 'w') as f:
  validation.to_csv(f, index=False)

Training data head


f0        f4        f5        f6        f7        f8        f9  \
31766 -0.156896 -0.026570  0.126073 -0.003524 -0.008004 -0.003524 -0.051983   
46080 -0.156896 -0.024665  0.419602 -0.003524 -0.008004 -0.003524 -0.051983   
87575 -0.156896 -0.027061 -0.076651 -0.003524 -0.008004 -0.003524 -0.051983   
16899 -0.156896 -0.025185 -0.017329 -0.003524 -0.008004 -0.003524 -0.051983   
45096 -0.156896 -0.024665 -0.049083 -0.003524 -0.008004 -0.003524 -0.051983   

            f10       f11      f12  ...     f3OTH     f3REJ    f3RSTO  \
31766 -0.007992  0.661505 -0.00648  ... -0.003524 -0.242059 -0.028203   
46080 -0.007992  0.661505 -0.00648  ... -0.003524 -0.242059 -0.028203   
87575 -0.007992  0.661505 -0.00648  ... -0.003524 -0.242059 -0.028203   
16899 -0.007992  0.661505 -0.00648  ... -0.003524 -0.242059 -0.028203   
45096 -0.007992  0.661505 -0.00648  ... -0.003524 -0.242059 -0.028203   

         f3RSTR      f3S0      f3S1      f3S2     f3S3      f3SF  target  
31766 -0.018981 -0.085028 -0.023908 -0.012208 -0.00788  0.262528       0  
46080 -0.018981 -0.085028 -0.023908 -0.012208 -0.00788  0.262528       0  
87575 -0.018981 -0.085028 -0.023908 -0.012208 -0.00788  0.262528       0  
16899 -0.018981 -0.085028 -0.023908 -0.012208 -0.00788  0.262528       0  
45096 -0.018981 -0.085028 -0.023908 -0.012208 -0.00788  0.262528       0  

[5 rows x 99 columns]

Copy the data in bucket ...


## Cloud training

### Accelerator and distribution support

| GPU | Multi-GPU Node | TPU | Workers | Parameter Server |
|---|---|---|---|---|
| Yes | Yes | No | Yes | Yes |

To have distribution and/or accelerators to your AI Platform training call, use parameters similar to the examples as shown below.
```bash
    --master-machine-type standard_gpu
    --worker-machine-type standard_gpu \
    --worker-count 2 \
    --parameter-server-machine-type standard \
    --parameter-server-count 1 \
```
### AI Platform training

- [Tabular Anomaly Detection ML container documentation](https://aihub.cloud.google.com/u/0/p/products%2F6427563a-f2a8-4f9e-a104-a4dbc95d4e3e).
- [AI Platform training documentation](https://cloud.google.com/sdk/gcloud/reference/ai-platform/jobs/submit/training).

In [ ]:
output_location = os.path.join('gs://', BUCKET_NAME, 'output')

job_name = "tabular_anomaly_detection_{}".format(time.strftime("%Y%m%d%H%M%S"))
!gcloud ai-platform jobs submit training $job_name \
    --master-image-uri gcr.io/aihub-c2t-containers/kfp-components/oob_algorithm/tabular_anomaly_detection:latest \
    --region $REGION \
    --scale-tier CUSTOM \
    --master-machine-type standard \
    -- \
    --output-location {output_location} \
    --training-data {training_data} \
    --validation-data {validation_data} \
    --target-column target \
    --data-type csv \
    --fresh-start True \
    --training-epochs 5

### Local training snippet

Note that the training can also be done locally with Docker
```bash
docker run \
    -v /tmp:/tmp \
    -it gcr.io/aihub-c2t-containers/kfp-components/oob_algorithm/tabular_anomaly_detection:latest \
    --output-location /tmp/fm_classification \
    --training-data /tmp/iris_train.csv \
    --validation-data /tmp/iris_valid.csv \
    --target-column target \
    --data-type csv \
    --training-epochs 10
```

## Monitor the training with TensorBoard

In [ ]:
try:
  %load_ext tensorboard
  %tensorboard --logdir {output_location}
except:
  !tensorboard --logdir {output_location}

## Inspect the Run Report

The "Run Report" will help you identify if the model was successfully trained.

In [19]:
if not tf.io.gfile.exists(os.path.join(output_location, 'report.html')):
  raise RuntimeError('The file report.html was not found. Did the training job finish?')

with tf.io.gfile.GFile(os.path.join(output_location, 'report.html')) as f:
  display(HTML(f.read()))

## Deployment parameters

In [20]:
#@markdown ---
model = 'anomaly_detection' #@param {type:"string"}
version = 'v5' #@param {type:"string"}
#@markdown ---

## Deploy the model for serving
- https://cloud.google.com/ai-platform/prediction/docs/deploying-models

In [21]:
# the exact location of the model is in model_uri.txt
with tf.io.gfile.GFile(os.path.join(output_location, 'model_uri.txt')) as f:
  model_uri = f.read()

# create a model
!gcloud ai-platform models create $model --regions $REGION

# create a version
!gcloud ai-platform versions create $version \
  --model $model \
  --runtime-version 1.15 \
  --origin $model_uri \
  --project $PROJECT_ID

ERROR: (gcloud.ai-platform.models.create) Resource in project [aihub-content-external] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name
Creating version (this might take a few minutes)......done.                    


## Use the endpoint for online predictions

In [22]:
validation_targets = validation['target']

# format the data for serving
instances = validation.iloc[:100].drop(columns='target').to_dict(orient='records')
display(instances[:1])

service = discovery.build('ml', 'v1')
name = 'projects/{project}/models/{model}/versions/{version}'.format(project=PROJECT_ID,
                                                                     model=model,
                                                                     version=version)
body = {'instances': instances}

response = service.projects().predict(name=name, body=body).execute()
if 'error' in response:
    raise RuntimeError(response['error'])

error_z_score = [row['error_z_score'] for row in response['predictions']]

result = pd.DataFrame()
result['predicted_is_anomaly'] = np.array(error_z_score) > 3.5
result['data_is_anomaly'] = validation_targets.iloc[:len(instances)].values == 1
result.head()

[{'f0': -0.15689603826360893,
  'f4': -0.025704163851021037,
  'f5': -0.0797828517984638,
  'f6': -0.0035240116048975505,
  'f7': -0.00800358902857826,
  'f8': -0.003524011604898069,
  'f9': -0.05198253475342395,
  'f10': -0.00799191272681093,
  'f11': 0.6615054117038175,
  'f12': -0.00648025473873549,
  'f13': -0.01536252761667093,
  'f14': -0.008189286700294809,
  'f15': -0.013330689656676883,
  'f16': -0.022828910812516767,
  'f17': -0.021440519066378394,
  'f18': -0.06331603295124155,
  'f19': 0.0,
  'f20': 0.0,
  'f21': -0.06104892691915754,
  'f22': -0.24602909760517405,
  'f23': -0.2641962225341414,
  'f24': -0.09584599165349855,
  'f25': -0.0987194213709275,
  'f26': -0.24550596602717586,
  'f27': -0.24794169791019005,
  'f28': 0.177332320380092,
  'f29': -0.15871193507785517,
  'f30': -0.47228251542263483,
  'f31': -0.9724412582981401,
  'f32': 0.6087419964027544,
  'f33': 0.5101486255686911,
  'f34': -0.3108008889283201,
  'f35': -0.43614622010394766,
  'f36': 0.3363872379300

/Users/evo/Library/Python/3.7/lib/python/site-packages/google/auth/_default.py:66: UserWarning:

Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/



predicted_is_anomaly  data_is_anomaly
0                 False            False
1                 False            False
2                 False            False
3                 False            False
4                 False            False

## Inspect the ML model

- [What if tool home page](https://pair-code.github.io/what-if-tool/index.html)
- [Installation](https://github.com/pair-code/what-if-tool#how-do-i-enable-it-for-use-in-a-jupyter-notebook)

In [ ]:
import witwidget
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

def adjust_prediction(pred):
  if pred > 3.5:
    return [0., 1.]
  else:
    return [1., 0]

config_builder = WitConfigBuilder(examples=validation.iloc[:100].to_dict(orient='records'))
config_builder.set_ai_platform_model(project=PROJECT_ID,
                                     model=model,
                                     version=version,
                                     adjust_prediction=adjust_prediction)
config_builder.set_model_type('classification')
config_builder.set_target_feature('target')
WitWidget(config_builder)

# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

In [ ]:
# Delete model version resource
! gcloud ai-platform versions delete $version --quiet --model $model 

# Delete model resource
! gcloud ai-platform models delete $model --quiet

# If training job is still running, cancel it
! gcloud ai-platform jobs cancel $job_name --quiet

# Delete Cloud Storage objects that were created
! gsutil -m rm -r $BUCKET_NAME